In [5]:
import pandas as pd
from neo4j import GraphDatabase
import os
import numpy as np

class Neo4jIngester:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def close(self):
        self.driver.close()
    
    def run_query(self, query, parameters=None):
        with self.driver.session() as session:
            result = session.run(query, parameters)
            return result

def clean_dataframe(df, id_cols):
    """Clean dataframe by removing rows with null/NaN in ID columns"""
    initial_count = len(df)
    df = df.dropna(subset=id_cols)
    removed = initial_count - len(df)
    if removed > 0:
        print(f"  ⚠️  Removed {removed} rows with null IDs")
    
    # Convert ID columns to string and strip whitespace
    for col in id_cols:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip()
    
    return df

def ingest_data():
    # Neo4j connection parameters - UPDATE THESE
    NEO4J_URI = "bolt://localhost:7687"
    NEO4J_USER = "neo4j" 
    NEO4J_PASSWORD = "password"  # CHANGE THIS
    
    print("=" * 80)
    print("🚀 STARTING NEO4J DATA INGESTION")
    print("=" * 80)
    
    ingester = Neo4jIngester(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
    
    # Clear existing data
    print("\n🗑️  Clearing existing data...")
    ingester.run_query("MATCH (n) DETACH DELETE n")
    print("  ✅ Database cleared")
    
    folder_path = 'output_csv/'
    
    # ==================================================================
    # 1. INGEST NODE ENTITIES
    # ==================================================================
    
    print("\n" + "=" * 80)
    print("PHASE 1: INGESTING NODES")
    print("=" * 80)
    
    # Companies
    print("\n🏢 Ingesting Companies...")
    companies_df = pd.read_csv(os.path.join(folder_path, 'companies.csv'))
    companies_df = clean_dataframe(companies_df, ['company_id'])
    query = """
    UNWIND $rows as row
    MERGE (c:Company {company_id: row.company_id})
    SET c.name = row.name,
        c.country = row.country,
        c.sector = row.sector,
        c.risk_score = toFloat(row.risk_score),
        c.opportunity_score = toFloat(row.opportunity_score),
        c.is_supplier = CASE WHEN row.is_supplier = 'True' THEN true ELSE false END
    """
    ingester.run_query(query, {'rows': companies_df.to_dict('records')})
    print(f"  ✅ {len(companies_df)} companies ingested")
    
    # Auditors
    print("\n👮 Ingesting Auditors...")
    auditors_df = pd.read_csv(os.path.join(folder_path, 'auditors.csv'))
    auditors_df = clean_dataframe(auditors_df, ['auditor_id'])
    query = """
    UNWIND $rows as row
    MERGE (a:Auditor {auditor_id: row.auditor_id})
    SET a.name = row.name,
        a.risk_level = row.risk_level
    """
    ingester.run_query(query, {'rows': auditors_df.to_dict('records')})
    print(f"  ✅ {len(auditors_df)} auditors ingested")
    
    # Shareholders
    print("\n💼 Ingesting Shareholders...")
    shareholders_df = pd.read_csv(os.path.join(folder_path, 'shareholders.csv'))
    shareholders_df = clean_dataframe(shareholders_df, ['shareholder_id'])
    query = """
    UNWIND $rows as row
    MERGE (s:Shareholder {shareholder_id: row.shareholder_id})
    SET s.name = row.name,
        s.type = row.type,
        s.risk_score = toFloat(row.risk_score),
        s.opportunity_score = toFloat(row.opportunity_score)
    """
    ingester.run_query(query, {'rows': shareholders_df.to_dict('records')})
    print(f"  ✅ {len(shareholders_df)} shareholders ingested")
    
    # Invoices
    print("\n📄 Ingesting Invoices...")
    invoices_df = pd.read_csv(os.path.join(folder_path, 'invoices.csv'))
    invoices_df = clean_dataframe(invoices_df, ['invoice_id'])
    query = """
    UNWIND $rows as row
    MERGE (i:Invoice {invoice_id: row.invoice_id})
    SET i.amount = toFloat(row.amount),
        i.status = row.status,
        i.is_simulated = CASE WHEN row.is_simulated = 'True' THEN true ELSE false END
    """
    ingester.run_query(query, {'rows': invoices_df.to_dict('records')})
    print(f"  ✅ {len(invoices_df)} invoices ingested")
    
    # ==================================================================
    # 2. INGEST RELATIONSHIPS - USING CORRECT COLUMN NAMES
    # ==================================================================
    
    print("\n" + "=" * 80)
    print("PHASE 2: CREATING RELATIONSHIPS")
    print("=" * 80)
    
    # OWNS_SHARE (shareholder_id, company_id, percentage)
    print("\n📈 Creating OWNS_SHARE relationships...")
    owns_share_df = pd.read_csv(os.path.join(folder_path, 'owns_share.csv'))
    owns_share_df = clean_dataframe(owns_share_df, ['shareholder_id', 'company_id'])
    query = """
    UNWIND $rows as row
    MATCH (s:Shareholder {shareholder_id: row.shareholder_id})
    MATCH (c:Company {company_id: row.company_id})
    MERGE (s)-[r:OWNS_SHARE]->(c)
    SET r.percentage = toFloat(row.percentage)
    """
    ingester.run_query(query, {'rows': owns_share_df.to_dict('records')})
    print(f"  ✅ {len(owns_share_df)} OWNS_SHARE relationships created")
    
    # AUDITED_BY (company_id, auditor_id)
    print("\n🔍 Creating AUDITED_BY relationships...")
    audited_by_df = pd.read_csv(os.path.join(folder_path, 'audited_by.csv'))
    audited_by_df = clean_dataframe(audited_by_df, ['company_id', 'auditor_id'])
    query = """
    UNWIND $rows as row
    MATCH (c:Company {company_id: row.company_id})
    MATCH (a:Auditor {auditor_id: row.auditor_id})
    MERGE (c)-[r:AUDITED_BY]->(a)
    """
    ingester.run_query(query, {'rows': audited_by_df.to_dict('records')})
    print(f"  ✅ {len(audited_by_df)} AUDITED_BY relationships created")
    
    # ISSUES_TO (company_id, invoice_id)
    print("\n📤 Creating ISSUES_TO relationships...")
    issues_to_df = pd.read_csv(os.path.join(folder_path, 'issues_to.csv'))
    issues_to_df = clean_dataframe(issues_to_df, ['company_id', 'invoice_id'])
    query = """
    UNWIND $rows as row
    MATCH (c:Company {company_id: row.company_id})
    MATCH (i:Invoice {invoice_id: row.invoice_id})
    MERGE (c)-[r:ISSUES_TO]->(i)
    """
    ingester.run_query(query, {'rows': issues_to_df.to_dict('records')})
    print(f"  ✅ {len(issues_to_df)} ISSUES_TO relationships created")
    
    # PAYS (company_id, invoice_id)
    print("\n💳 Creating PAYS relationships...")
    pays_df = pd.read_csv(os.path.join(folder_path, 'pays.csv'))
    pays_df = clean_dataframe(pays_df, ['company_id', 'invoice_id'])
    query = """
    UNWIND $rows as row
    MATCH (c:Company {company_id: row.company_id})
    MATCH (i:Invoice {invoice_id: row.invoice_id})
    MERGE (c)-[r:PAYS]->(i)
    """
    ingester.run_query(query, {'rows': pays_df.to_dict('records')})
    print(f"  ✅ {len(pays_df)} PAYS relationships created")
    
    # SUBSIDIARY_OF (child_company_id, parent_company_id, since_year)
    print("\n🏢 Creating SUBSIDIARY_OF relationships...")
    subsidiary_df = pd.read_csv(os.path.join(folder_path, 'subsidiary_of.csv'))
    print(f"  📋 Columns in subsidiary_of.csv: {list(subsidiary_df.columns)}")
    subsidiary_df = clean_dataframe(subsidiary_df, ['child_company_id', 'parent_company_id'])
    query = """
    UNWIND $rows as row
    MATCH (child:Company {company_id: row.child_company_id})
    MATCH (parent:Company {company_id: row.parent_company_id})
    MERGE (child)-[r:SUBSIDIARY_OF]->(parent)
    SET r.since_year = toInteger(row.since_year)
    """
    ingester.run_query(query, {'rows': subsidiary_df.to_dict('records')})
    print(f"  ✅ {len(subsidiary_df)} SUBSIDIARY_OF relationships created")
    
    # SUPPLIES (supplier_company_id, buyer_company_id, annual_volume)
    print("\n🚚 Creating SUPPLIES relationships...")
    supplies_df = pd.read_csv(os.path.join(folder_path, 'supplies.csv'))
    print(f"  📋 Columns in supplies.csv: {list(supplies_df.columns)}")
    supplies_df = clean_dataframe(supplies_df, ['supplier_company_id', 'buyer_company_id'])
    query = """
    UNWIND $rows as row
    MATCH (supplier:Company {company_id: row.supplier_company_id})
    MATCH (buyer:Company {company_id: row.buyer_company_id})
    MERGE (supplier)-[r:SUPPLIES]->(buyer)
    SET r.annual_volume = toFloat(row.annual_volume)
    """
    ingester.run_query(query, {'rows': supplies_df.to_dict('records')})
    print(f"  ✅ {len(supplies_df)} SUPPLIES relationships created")
    
    # ==================================================================
    # 3. CREATE INDEXES FOR PERFORMANCE
    # ==================================================================
    
    print("\n" + "=" * 80)
    print("PHASE 3: CREATING INDEXES")
    print("=" * 80)
    
    indexes = [
        "CREATE INDEX company_id_idx IF NOT EXISTS FOR (c:Company) ON (c.company_id)",
        "CREATE INDEX shareholder_id_idx IF NOT EXISTS FOR (s:Shareholder) ON (s.shareholder_id)",
        "CREATE INDEX auditor_id_idx IF NOT EXISTS FOR (a:Auditor) ON (a.auditor_id)",
        "CREATE INDEX invoice_id_idx IF NOT EXISTS FOR (i:Invoice) ON (i.invoice_id)",
        "CREATE INDEX company_risk_idx IF NOT EXISTS FOR (c:Company) ON (c.risk_score)",
        "CREATE INDEX auditor_risk_idx IF NOT EXISTS FOR (a:Auditor) ON (a.risk_level)"
    ]
    
    for idx_query in indexes:
        try:
            ingester.run_query(idx_query)
            print(f"  ✅ {idx_query.split('FOR')[0].split('IF')[0].strip()}")
        except Exception as e:
            print(f"  ⚠️  Index creation skipped (may already exist)")
    
    # ==================================================================
    # 4. GRAPH SUMMARY
    # ==================================================================
    
    print("\n" + "=" * 80)
    print("📊 GRAPH SUMMARY")
    print("=" * 80)
    
    # Count nodes by label
    with ingester.driver.session() as session:
        # Node counts
        node_query = """
        MATCH (n)
        RETURN labels(n)[0] as label, count(n) as count
        ORDER BY count DESC
        """
        result = session.run(node_query)
        print("\nNodes:")
        for record in result:
            print(f"  • {record['label']}: {record['count']:,}")
        
        # Relationship counts
        rel_query = """
        MATCH ()-[r]->()
        RETURN type(r) as type, count(r) as count
        ORDER BY count DESC
        """
        result = session.run(rel_query)
        print("\nRelationships:")
        for record in result:
            print(f"  • {record['type']}: {record['count']:,}")
        
        # Total counts
        total_query = """
        MATCH (n)
        WITH count(n) as nodeCount
        MATCH ()-[r]->()
        RETURN nodeCount, count(r) as relCount
        """
        result = session.run(total_query)
        record = result.single()
        print(f"\n{'=' * 80}")
        print(f"TOTALS: {record['nodeCount']:,} nodes, {record['relCount']:,} relationships")
        print(f"{'=' * 80}")
    
    # ==================================================================
    # 5. VALIDATION QUERIES
    # ==================================================================
    
    print("\n" + "=" * 80)
    print("🔍 VALIDATION CHECKS")
    print("=" * 80)
    
    with ingester.driver.session() as session:
        # Check for high-risk auditors
        result = session.run("""
            MATCH (a:Auditor)
            WHERE a.risk_level = 'HIGH'
            RETURN count(a) as count
        """)
        high_risk = result.single()['count']
        print(f"\n✓ High-risk auditors: {high_risk}")
        
        # Check for companies with subsidiaries
        result = session.run("""
            MATCH (c:Company)-[:SUBSIDIARY_OF]->()
            RETURN count(DISTINCT c) as count
        """)
        subs = result.single()['count']
        print(f"✓ Companies with parent companies: {subs}")
        
        # Check for supply relationships
        result = session.run("""
            MATCH ()-[r:SUPPLIES]->()
            RETURN count(r) as count
        """)
        supplies = result.single()['count']
        print(f"✓ Supply relationships: {supplies}")
        
        # Check for circular patterns (sample)
        result = session.run("""
            MATCH path = (c1:Company)-[:SUPPLIES]->(c2:Company)-[:SUPPLIES]->(c3:Company)-[:SUPPLIES]->(c1)
            RETURN count(path) as count
            LIMIT 100
        """)
        cycles = result.single()['count']
        print(f"✓ Sample circular supply patterns found: {cycles}")
        
        # Check for ownership concentration
        result = session.run("""
            MATCH (s:Shareholder)-[o:OWNS_SHARE]->(c:Company)
            WHERE o.percentage > 25
            RETURN count(o) as count
        """)
        major_stakes = result.single()['count']
        print(f"✓ Ownership stakes >25%: {major_stakes}")
    
    print("\n" + "=" * 80)
    print("🎉 INGESTION COMPLETE!")
    print("=" * 80)
    print("\nNext steps:")
    print("  1. Open Neo4j Browser at http://localhost:7474")
    print("  2. Run: CALL db.schema.visualization()")
    print("  3. Start detecting fraud patterns!")
    print("=" * 80)
    
    ingester.close()

if __name__ == "__main__":
    try:
        ingest_data()
    except Exception as e:
        print(f"\n❌ ERROR: {str(e)}")
        print("\nMake sure:")
        print("  1. Neo4j is running (check localhost:7474)")
        print("  2. Credentials are correct")
        print("  3. CSV files exist in output_csv/ folder")
        import traceback
        traceback.print_exc()

🚀 STARTING NEO4J DATA INGESTION

🗑️  Clearing existing data...
  ✅ Database cleared

PHASE 1: INGESTING NODES

🏢 Ingesting Companies...
  ✅ 500 companies ingested

👮 Ingesting Auditors...
  ✅ 10 auditors ingested

💼 Ingesting Shareholders...
  ✅ 250 shareholders ingested

📄 Ingesting Invoices...
  ✅ 1500 invoices ingested

PHASE 2: CREATING RELATIONSHIPS

📈 Creating OWNS_SHARE relationships...
  ✅ 1777 OWNS_SHARE relationships created

🔍 Creating AUDITED_BY relationships...
  ✅ 500 AUDITED_BY relationships created

📤 Creating ISSUES_TO relationships...
  ✅ 1500 ISSUES_TO relationships created

💳 Creating PAYS relationships...
  ✅ 1490 PAYS relationships created

🏢 Creating SUBSIDIARY_OF relationships...
  📋 Columns in subsidiary_of.csv: ['child_company_id', 'parent_company_id', 'since_year']
  ✅ 224 SUBSIDIARY_OF relationships created

🚚 Creating SUPPLIES relationships...
  📋 Columns in supplies.csv: ['supplier_company_id', 'buyer_company_id', 'annual_volume']
  ✅ 359 SUPPLIES relation

In [3]:
import pandas as pd
from neo4j import GraphDatabase
import os
import numpy as np

class Neo4jIngester:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def close(self):
        self.driver.close()
    
    def run_query(self, query, parameters=None):
        with self.driver.session() as session:
            session.run(query, parameters)

def clean_dataframe(df, id_cols):
    """Clean dataframe by removing rows with null/NaN in ID columns"""
    initial_count = len(df)
    df = df.dropna(subset=id_cols)
    print(f"  Removed {initial_count - len(df)} rows with null IDs")
    
    # Convert ID columns to string
    for col in id_cols:
        if col in df.columns:
            df[col] = df[col].astype(str)
    
    return df

def ingest_data():
    # Neo4j connection parameters - UPDATE THESE
    NEO4J_URI = "bolt://localhost:7687"
    NEO4J_USER = "neo4j" 
    NEO4J_PASSWORD = "password"
    
    ingester = Neo4jIngester(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
    
    # Clear existing data
    print("🗑️  Clearing existing data...")
    ingester.run_query("MATCH (n) DETACH DELETE n")
    
    folder_path = 'output_csv/'
    
    # 1. INGEST NODES - Using EXACT CSV column names
    
    # Companies
    print("\n🏢 Ingesting Companies...")
    companies_df = pd.read_csv(os.path.join(folder_path, 'companies.csv'))
    companies_df = clean_dataframe(companies_df, ['company_id'])
    query = """
    UNWIND $rows as row
    MERGE (c:Company {company_id: row.company_id})
    SET c.name = row.name,
        c.country = row.country,
        c.sector = row.sector,
        c.risk_score = toFloat(row.risk_score),
        c.opportunity_score = toFloat(row.opportunity_score),
        c.is_supplier = row.is_supplier = 'True'
    """
    ingester.run_query(query, {'rows': companies_df.to_dict('records')})
    print(f"  ✅ {len(companies_df)} companies ingested")
    
    # Auditors
    print("\n👮 Ingesting Auditors...")
    auditors_df = pd.read_csv(os.path.join(folder_path, 'auditors.csv'))
    auditors_df = clean_dataframe(auditors_df, ['auditor_id'])
    query = """
    UNWIND $rows as row
    MERGE (a:Auditor {auditor_id: row.auditor_id})
    SET a.name = row.name,
        a.risk_level = row.risk_level
    """
    ingester.run_query(query, {'rows': auditors_df.to_dict('records')})
    print(f"  ✅ {len(auditors_df)} auditors ingested")
    
    # Shareholders
    print("\n💼 Ingesting Shareholders...")
    shareholders_df = pd.read_csv(os.path.join(folder_path, 'shareholders.csv'))
    shareholders_df = clean_dataframe(shareholders_df, ['shareholder_id'])
    query = """
    UNWIND $rows as row
    MERGE (s:Shareholder {shareholder_id: row.shareholder_id})
    SET s.name = row.name,
        s.type = row.type,
        s.risk_score = toFloat(row.risk_score),
        s.opportunity_score = toFloat(row.opportunity_score)
    """
    ingester.run_query(query, {'rows': shareholders_df.to_dict('records')})
    print(f"  ✅ {len(shareholders_df)} shareholders ingested")
    
    # Invoices
    print("\n📄 Ingesting Invoices...")
    invoices_df = pd.read_csv(os.path.join(folder_path, 'invoices.csv'))
    invoices_df = clean_dataframe(invoices_df, ['invoice_id'])
    query = """
    UNWIND $rows as row
    MERGE (i:Invoice {invoice_id: row.invoice_id})
    SET i.amount = toFloat(row.amount),
        i.status = row.status,
        i.is_simulated = row.is_simulated = 'True'
    """
    ingester.run_query(query, {'rows': invoices_df.to_dict('records')})
    print(f"  ✅ {len(invoices_df)} invoices ingested")
    
    # 2. INGEST RELATIONSHIPS - Using EXACT CSV column names
    
    # owns_share
    print("\n📈 Creating OWNS_SHARE relationships...")
    owns_share_df = pd.read_csv(os.path.join(folder_path, 'owns_share.csv'))
    owns_share_df = clean_dataframe(owns_share_df, ['shareholder_id', 'company_id'])
    query = """
    UNWIND $rows as row
    MATCH (s:Shareholder {shareholder_id: row.shareholder_id})
    MATCH (c:Company {company_id: row.company_id})
    MERGE (s)-[r:OWNS_SHARE]->(c)
    SET r.shares = toFloat(coalesce(row.shares, 0)),
        r.percentage = toFloat(coalesce(row.percentage, 0))
    """
    ingester.run_query(query, {'rows': owns_share_df.to_dict('records')})
    print(f"  ✅ {len(owns_share_df)} OWNS_SHARE relationships")
    
    # audited_by
    print("🔍 Creating AUDITED_BY relationships...")
    audited_by_df = pd.read_csv(os.path.join(folder_path, 'audited_by.csv'))
    audited_by_df = clean_dataframe(audited_by_df, ['company_id', 'auditor_id'])
    query = """
    UNWIND $rows as row
    MATCH (c:Company {company_id: row.company_id})
    MATCH (a:Auditor {auditor_id: row.auditor_id})
    MERGE (c)-[r:AUDITED_BY]->(a)
    SET r.since_year = toInteger(coalesce(row.since_year, 0))
    """
    ingester.run_query(query, {'rows': audited_by_df.to_dict('records')})
    print(f"  ✅ {len(audited_by_df)} AUDITED_BY relationships")
    
    # issues_to
    print("📤 Creating ISSUES_TO relationships...")
    issues_to_df = pd.read_csv(os.path.join(folder_path, 'issues_to.csv'))
    issues_to_df = clean_dataframe(issues_to_df, ['company_id', 'invoice_id'])
    query = """
    UNWIND $rows as row
    MATCH (c:Company {company_id: row.company_id})
    MATCH (i:Invoice {invoice_id: row.invoice_id})
    MERGE (c)-[r:ISSUES_TO]->(i)
    """
    ingester.run_query(query, {'rows': issues_to_df.to_dict('records')})
    print(f"  ✅ {len(issues_to_df)} ISSUES_TO relationships")
    
    # pays
    print("💳 Creating PAYS relationships...")
    pays_df = pd.read_csv(os.path.join(folder_path, 'pays.csv'))
    pays_df = clean_dataframe(pays_df, ['company_id', 'invoice_id'])
    query = """
    UNWIND $rows as row
    MATCH (c:Company {company_id: row.company_id})
    MATCH (i:Invoice {invoice_id: row.invoice_id})
    MERGE (c)-[r:PAYS]->(i)
    """
    ingester.run_query(query, {'rows': pays_df.to_dict('records')})
    print(f"  ✅ {len(pays_df)} PAYS relationships")
    
    # subsidiary_of
    print("🏢 Creating SUBSIDIARY_OF relationships...")
    subsidiary_df = pd.read_csv(os.path.join(folder_path, 'subsidiary_of.csv'))
    subsidiary_df = clean_dataframe(subsidiary_df, ['company_id', 'company_id'])
    query = """
    UNWIND $rows as row
    MATCH (child:Company {company_id: row.company_id})
    MATCH (parent:Company {company_id: row.company_id})
    MERGE (child)-[r:SUBSIDIARY_OF]->(parent)
    SET r.since_year = toInteger(coalesce(row.since_year, 0))
    """
    ingester.run_query(query, {'rows': subsidiary_df.to_dict('records')})
    print(f"  ✅ {len(subsidiary_df)} SUBSIDIARY_OF relationships")
    
    # supplies
    print("🚚 Creating SUPPLIES relationships...")
    supplies_df = pd.read_csv(os.path.join(folder_path, 'supplies.csv'))
    supplies_df = clean_dataframe(supplies_df, ['company_id', 'company_id'])
    query = """
    UNWIND $rows as row
    MATCH (supplier:Company {company_id: row.company_id})
    MATCH (customer:Company {company_id: row.company_id})
    MERGE (supplier)-[r:SUPPLIES]->(customer)
    SET r.annual_volume = toFloat(coalesce(row.annual_volume, 0))
    """
    ingester.run_query(query, {'rows': supplies_df.to_dict('records')})
    print(f"  ✅ {len(supplies_df)} SUPPLIES relationships")
    
    print("\n🎉 COMPLETE INGESTION SUCCESSFUL!")
    
    # Graph Summary
    print("\n📊 GRAPH SUMMARY:")
    summary_query = """
    MATCH (n)
    RETURN apoc.label.names(n) as labels, count(n) as count
    ORDER BY count DESC
    """
    try:
        with ingester.driver.session() as session:
            result = session.run(summary_query)
            for record in result:
                print(f"  {record['labels']}: {record['count']}")
    except:
        # Fallback summary
        print("  Run `CALL db.labels()` and `CALL db.relationshipTypes()` in Neo4j Browser")
    
    ingester.close()

if __name__ == "__main__":
    ingest_data()


🗑️  Clearing existing data...

🏢 Ingesting Companies...
  Removed 0 rows with null IDs
  ✅ 500 companies ingested

👮 Ingesting Auditors...
  Removed 0 rows with null IDs
  ✅ 10 auditors ingested

💼 Ingesting Shareholders...
  Removed 0 rows with null IDs
  ✅ 250 shareholders ingested

📄 Ingesting Invoices...
  Removed 0 rows with null IDs
  ✅ 1500 invoices ingested

📈 Creating OWNS_SHARE relationships...
  Removed 0 rows with null IDs
  ✅ 1766 OWNS_SHARE relationships
🔍 Creating AUDITED_BY relationships...
  Removed 0 rows with null IDs
  ✅ 500 AUDITED_BY relationships
📤 Creating ISSUES_TO relationships...
  Removed 0 rows with null IDs
  ✅ 1500 ISSUES_TO relationships
💳 Creating PAYS relationships...
  Removed 0 rows with null IDs
  ✅ 1490 PAYS relationships
🏢 Creating SUBSIDIARY_OF relationships...
  Removed 0 rows with null IDs
  ✅ 215 SUBSIDIARY_OF relationships
🚚 Creating SUPPLIES relationships...
  Removed 0 rows with null IDs
  ✅ 362 SUPPLIES relationships

🎉 COMPLETE INGESTION 